In [ ]:
pip install easyocr pytesseract jiwer shapely opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 26.9 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import easyocr
import pytesseract
import numpy as np
from jiwer import wer, cer
from shapely.geometry import Polygon

In [ ]:
def parse_bbox_file(file_path):
    """
    Parse bbox txt file. Format: 8 numbers for polygon + text.
    Example line: x1,y1,x2,y2,x3,y3,x4,y4,text
    """
    bboxes, texts = [], []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(",")
            coords = list(map(int, parts[:8]))
            text = ",".join(parts[8:])  # text may contain commas
            bboxes.append(coords)
            texts.append(text)
    return bboxes, texts

In [ ]:
def polygon_iou(poly1, poly2):
    """Calculate IoU between two polygons (list of 8 points)."""
    poly1 = Polygon(np.array(poly1).reshape(-1, 2))
    poly2 = Polygon(np.array(poly2).reshape(-1, 2))
    if not poly1.is_valid or not poly2.is_valid:
        return 0.0
    inter = poly1.intersection(poly2).area
    union = poly1.union(poly2).area
    return inter / union if union > 0 else 0

In [ ]:
def rect_iou(rect1, rect2):
    """Calculate IoU between two rectangles [x_min, y_min, x_max, y_max]."""
    x1 = max(rect1[0], rect2[0])
    y1 = max(rect1[1], rect2[1])
    x2 = min(rect1[2], rect2[2])
    y2 = min(rect1[3], rect2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    rect1_area = (rect1[2] - rect1[0]) * (rect1[3] - rect1[1])
    rect2_area = (rect2[2] - rect2[0]) * (rect2[3] - rect2[1])
    union_area = rect1_area + rect2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

In [ ]:
def rect_from_polygon(poly):
    """Convert 8-point polygon to axis-aligned rectangle [x_min, y_min, x_max, y_max]."""
    pts = np.array(poly).reshape(-1, 2)
    x_min, y_min = pts[:, 0].min(), pts[:, 1].min()
    x_max, y_max = pts[:, 0].max(), pts[:, 1].max()
    return [x_min, y_min, x_max, y_max]


In [ ]:
def evaluate_easyocr(image_path, bbox_path, reader):
    gt_bboxes, gt_texts = parse_bbox_file(bbox_path)
    img = cv2.imread(image_path)

    results = reader.readtext(img)

    # Evaluate text accuracy
    pred_texts = [r[1] for r in results]
    cer_score = cer(" ".join(gt_texts), " ".join(pred_texts))
    wer_score = wer(" ".join(gt_texts), " ".join(pred_texts))

    # Evaluate bbox IoU (polygon vs polygon)
    ious = []
    for gt, pred in zip(gt_bboxes, results):
        pred_poly = np.array(pred[0]).flatten().tolist()
        ious.append(polygon_iou(gt, pred_poly))

    return cer_score, wer_score, np.mean(ious) if ious else 0

In [ ]:
def evaluate_tesseract(image_path, bbox_path):
    gt_bboxes, gt_texts = parse_bbox_file(bbox_path)
    img = cv2.imread(image_path)

    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

    pred_texts = [t for t in data["text"] if t.strip() != ""]
    cer_score = cer(" ".join(gt_texts), " ".join(pred_texts))
    wer_score = wer(" ".join(gt_texts), " ".join(pred_texts))

    pred_bboxes = []
    for i in range(len(data["text"])):
        if data["text"][i].strip() == "":
            continue
        x, y, w, h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
        pred_bboxes.append([x, y, x + w, y + h])

    # IoU (rectangles)
    ious = []
    for gt, pred in zip(gt_bboxes, pred_bboxes):
        gt_rect = rect_from_polygon(gt)
        ious.append(rect_iou(gt_rect, pred))

    return cer_score, wer_score, np.mean(ious) if ious else 0

In [ ]:
# --- Step 1: Mount Google Drive (optional) ---
from google.colab import drive
drive.mount('/content/drive')  # Uncomment if dataset is in Google Drive

KeyboardInterrupt: 

In [ ]:
images_folder = "/content/drive/MyDrive/img"
bboxes_folder = "/content/drive/MyDrive/box"

In [ ]:
reader = easyocr.Reader(["en"])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
easy_cer_list, easy_wer_list, easy_iou_list = [], [], []
tess_cer_list, tess_wer_list, tess_iou_list = [], [], []

image_files = [f for f in os.listdir(images_folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

for idx, img_file in enumerate(image_files, start=1):
    image_path = os.path.join(images_folder, img_file)
    bbox_path = os.path.join(bboxes_folder, os.path.splitext(img_file)[0] + ".csv")


    if not os.path.exists(bbox_path):
        continue

    # EasyOCR
    easy_cer, easy_wer, easy_iou = evaluate_easyocr(image_path, bbox_path, reader)
    easy_cer_list.append(easy_cer)
    easy_wer_list.append(easy_wer)
    easy_iou_list.append(easy_iou)

    # Tesseract
    tess_cer, tess_wer, tess_iou = evaluate_tesseract(image_path, bbox_path)
    tess_cer_list.append(tess_cer)
    tess_wer_list.append(tess_wer)
    tess_iou_list.append(tess_iou)

    # Progress update every 20 images
    if idx % 20 == 0:
        print(f"Processed {idx} / {len(image_files)} images...")

# --- Print mean results ---
print("\n=== Final Averages over Dataset ===")
print(f" EasyOCR → CER: {np.mean(easy_cer_list):.3f}, "
      f"WER: {np.mean(easy_wer_list):.3f}, "
      f"IoU: {np.mean(easy_iou_list):.3f}")

print(f" Tesseract → CER: {np.mean(tess_cer_list):.3f}, "
      f"WER: {np.mean(tess_wer_list):.3f}, "
      f"IoU: {np.mean(tess_iou_list):.3f}")


Processed 20 / 627 images...
Processed 40 / 627 images...
Processed 60 / 627 images...
Processed 80 / 627 images...
Processed 100 / 627 images...
Processed 120 / 627 images...
Processed 140 / 627 images...
Processed 160 / 627 images...
Processed 180 / 627 images...
Processed 200 / 627 images...
Processed 220 / 627 images...
Processed 240 / 627 images...
Processed 260 / 627 images...
Processed 280 / 627 images...
Processed 300 / 627 images...
Processed 320 / 627 images...
Processed 340 / 627 images...
Processed 360 / 627 images...
Processed 380 / 627 images...
Processed 400 / 627 images...
Processed 420 / 627 images...
Processed 460 / 627 images...
Processed 480 / 627 images...
Processed 500 / 627 images...
Processed 520 / 627 images...
Processed 540 / 627 images...
Processed 560 / 627 images...
Processed 580 / 627 images...
Processed 600 / 627 images...
Processed 620 / 627 images...

=== Final Averages over Dataset ===
 EasyOCR → CER: 0.407, WER: 0.741, IoU: 0.061
 Tesseract → CER: 0.4